In [1]:
import sys
sys.path.append('/Users/michele/code/micheledeantoni/APP/back-end/')


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from player_profiling import preprocessor


In [3]:
data = preprocessor.load_data_fifa23()


In [4]:
df =preprocessor.filter_data(data)
trans = preprocessor.preprocess(df)


/Users/michele/code/micheledeantoni/APP/back-end/player_profiling/preprocessor.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  characteristics['goalkeeping_speed'].fillna(0, inplace=True)


In [5]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(trans)


PCA()

### Alternative with less PCAs

## Try with tsne

In [6]:
from sklearn.manifold import TSNE
# Run t-SNE
tsne = TSNE(
    n_components=3,
    learning_rate='auto',
    perplexity=30,
    early_exaggeration=12,
    init='pca'
)
X_tsne = tsne.fit_transform(trans)
X_tsne


array([[-26.521366  ,  -0.07008406,  11.613367  ],
       [-27.949518  ,  -1.7409573 ,  16.272163  ],
       [-28.066256  ,  -1.56851   ,  16.308311  ],
       ...,
       [  3.8455694 , -22.688475  ,  -5.0702515 ],
       [  2.2495737 , -20.108086  , -13.406302  ],
       [ 27.943071  ,   4.85977   , -12.663315  ]], dtype=float32)

In [7]:
from sklearn.cluster import KMeans

# Fit K-means
km = KMeans(n_clusters=32)
km.fit(X_tsne)


/Users/michele/.pyenv/versions/3.10.6/envs/APP/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=32)

In [8]:
km.labels_


array([31, 31, 31, ..., 17, 17, 11], dtype=int32)

In [9]:
data['label']= km.labels_


In [10]:
def print_best_rows_by_label(dataframe):
    # Group the DataFrame by the 'label' column
    grouped_data = dataframe.groupby('label')

    # Iterate over groups
    for label, group in grouped_data:
        print(f"\nLabel: {label}")

        # Sort the rows within each group based on the 'overall' column
        sorted_group = group.sort_values(by='overall', ascending=False)

        # Select the top 5 rows
        top_rows = sorted_group.head(5)

        # Print the 'long_name' values for the selected rows
        for index, row in top_rows.iterrows():
            print(f"Index: {index}, Long Name: {row['long_name']}, position {row['player_positions']}")


# Call the function
print_best_rows_by_label(data)



Label: 0
Index: 468, Long Name: Mauro Wilney Arambarri Rosa, position CDM, CM
Index: 603, Long Name: Adnan Claudiano Vidual Machado, position RB
Index: 824, Long Name: 守田 英正, position CM, CDM
Index: 984, Long Name: Victorio Martin Maia Padilha Jabim, position RB, RM
Index: 986, Long Name: Murildo Kauê Cadete Teixeira, position LM

Label: 1
Index: 1019, Long Name: Musa Barrow, position CF, ST, LW
Index: 1347, Long Name: Arnaud Kalimuendo-Muinga, position ST
Index: 1353, Long Name: Hugo Ekitike, position ST
Index: 1234, Long Name: Adilson Patrick Edrada Pereira, position CAM
Index: 1648, Long Name: Estevão Daniel Formoso Serpa, position ST, CF

Label: 2
Index: 59016, Long Name: Mario Vušković, position CB
Index: 3697, Long Name: Lorenzo Tonelli, position CB
Index: 4058, Long Name: Erhan Mašović, position CB, CDM
Index: 4064, Long Name: Kennedy Kofi Boateng, position CB
Index: 4029, Long Name: Dara Joseph O'Shea, position CB, RB

Label: 3
Index: 853, Long Name: Héctor Alfredo Moreno Herr

In [15]:
data.to_csv('/Users/michele/code/micheledeantoni/APP/back-end/raw_data/labelled_data_v1.csv')


In [13]:
import numpy as np

# Assuming 'data' is your main DataFrame with k-means labels and 'Xproj3' is a separate DataFrame

# Step 1: Choose a Player
player_of_interest_index = 26  # Replace with the index of the player you're interested in

# Step 2: Get the Cluster Assignment of the Player of Interest
cluster_of_interest = data.loc[player_of_interest_index, 'label']

# Step 3: Filter Data for the Same Cluster
cluster_indices = np.where(data['label'] == cluster_of_interest)[0]

# Step 4: Calculate Distances
distances = np.linalg.norm(X_tsne[cluster_indices] - X_tsne[player_of_interest_index], axis=1)

# Step 5: Identify the Closest Player
# Step 5: Identify the 5 Closest Players
closest_player_indices = cluster_indices[np.argsort(distances)[1:6]]
closest_players = data.iloc[closest_player_indices]

print(f"The closest player to the player of interest is:\n{closest_players['long_name']}")


The closest player to the player of interest is:
1081              Fredrik Midtsjø
130159     Allan Marques Loureiro
1664      Rodrigo Germán Aliendro
173                 Konrad Laimer
1584               Dorukhan Toköz
Name: long_name, dtype: object


In [141]:
import pandas as pd
from fuzzywuzzy import process

def find_best_match(data, column_name='long_name'):
    # Ask for user input
    user_input = input("Enter the player's name: ")

    # Get a list of possible matches and their scores
    possibilities = process.extract(user_input, data[column_name], limit=5)

    # Display the possibilities to the user
    print("Possible matches:")
    for i, (match, score) in enumerate(possibilities):
        print(f"{i + 1}. {match} (Score: {score})")

    # If there's only one possibility or a clear winner, return it
    if possibilities[0][1] > 90:
        return possibilities[0][0]

    # Ask the user to choose from the list if there are multiple possibilities
    user_choice = input("Enter the number corresponding to the correct player: ")
    chosen_index = int(user_choice) - 1

    # Return the chosen player
    return possibilities[chosen_index][0]

# Example usage:
# Assuming 'data' is your DataFrame with a 'long_name' column
# best_match = find_best_match(data)
# print(f"The best match is: {best_match}")


/Users/michele/.pyenv/versions/3.10.6/envs/APP/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [142]:
find_best_match(data)


In [ ]:
import pandas as pd
from fuzzywuzzy import process
from multiprocessing import cpu_count

def find_best_match(data, column_name='long_name'):
    # Ask for user input
    user_input = input("Enter the player's name: ")

    # Get a list of possible matches and their scores using parallel processing
    possibilities = process.extract(user_input, data[column_name], limit=5, scorer=process.extract, processor=cpu_count())

    # Display the possibilities to the user
    print("Possible matches:")
    for i, (match, score) in enumerate(possibilities):
        print(f"{i + 1}. {match} (Score: {score})")

    # If there's only one possibility or a clear winner, return it
    if possibilities[0][1] > 90:
        return possibilities[0][0]

    # Ask the user to choose from the list if there are multiple possibilities
    user_choice = input("Enter the number corresponding to the correct player: ")
    chosen_index = int(user_choice) - 1

    # Return the chosen player
    return possibilities[chosen_index][0]

# Example usage:
# Assuming 'data' is your DataFrame with a 'long_name' column
# best_match = find_best_match(data)
# print(f"The best match is: {best_match}")
